## Creating the MongoDB Database

In [1]:
!pip install pymongo

Error processing line 7 of C:\Users\annag\Anaconda3\lib\site-packages\pywin32.pth:

  Traceback (most recent call last):
    File "C:\Users\annag\Anaconda3\lib\site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
  ModuleNotFoundError: No module named 'pywin32_bootstrap'

Remainder of file ignored


In [2]:
import pymongo

Start the mongo daemon!

In [3]:
from pymongo import MongoClient
import pandas as pd

In [ ]:
#client = MongoClient('localhost', 27017)
db = client.project2 #Create the database

In [ ]:
#Create the Group Member collection
collection = db.groupMember